In [2]:
# ==============================================================================
# 1. SETUP - INSTALL AND IMPORT LIBS
# ==============================================================================
!pip install langgraph langchain_core==0.1.52 --quiet

from langgraph.graph import StateGraph, START, END
from typing import TypedDict
import time

# ==============================================================================
# 2. DEFINE THE STATE AND WORKFLOW LOGIC
# ==============================================================================
class StreamingState(TypedDict):
    input: str
    result_a: str
    result_b: str
    final_result: str

def process_a(state: StreamingState) -> dict:
    print("  (Executing Node A...)")
    time.sleep(1)
    return {"result_a": f"A({state['input']})"}

def process_b(state: StreamingState) -> dict:
    print("  (Executing Node B...)")
    time.sleep(1)
    return {"result_b": f"B({state['result_a']})"}

def final_process(state: StreamingState) -> dict:
    print("  (Executing Final Node...)")
    time.sleep(1)
    return {"final_result": f"Final({state['result_b']})"}

# ==============================================================================
# 3. BUILD AND COMPILE THE WORKFLOW
# ==============================================================================
def create_streaming_workflow():
    workflow = StateGraph(StreamingState)
    workflow.add_node("A", process_a)
    workflow.add_node("B", process_b)
    workflow.add_node("Final", final_process)
    workflow.add_edge(START, "A")
    workflow.add_edge("A", "B")
    workflow.add_edge("B", "Final")
    workflow.add_edge("Final", END)
    return workflow.compile()

# ==============================================================================
# 4. RUN THE WORKFLOW AND DEMONSTRATE STREAMING
# ==============================================================================
if __name__ == "__main__":
    app = create_streaming_workflow()
    initial_input = {"input": "start"}

    print("--- Part 1: Streaming results as they are produced using .stream() ---")
    # .stream() yields the output of each node as it completes.
    for chunk in app.stream(initial_input):
        node, output = next(iter(chunk.items()))
        print(f"Streaming Output from Node '{node}':")
        print(f"  {output}\n")

    print("\n--- Part 2: For comparison, the standard .invoke() method ---")
    # .invoke() blocks until the entire graph is finished and only returns the final state.
    final_output = app.invoke(initial_input)
    print("Invoke only returns the complete, final state:")
    print(f"  {final_output}")

--- Part 1: Streaming results as they are produced using .stream() ---
  (Executing Node A...)
Streaming Output from Node 'A':
  {'result_a': 'A(start)'}

  (Executing Node B...)
Streaming Output from Node 'B':
  {'result_b': 'B(A(start))'}

  (Executing Final Node...)
Streaming Output from Node 'Final':
  {'final_result': 'Final(B(A(start)))'}


--- Part 2: For comparison, the standard .invoke() method ---
  (Executing Node A...)
  (Executing Node B...)
  (Executing Final Node...)
Invoke only returns the complete, final state:
  {'input': 'start', 'result_a': 'A(start)', 'result_b': 'B(A(start))', 'final_result': 'Final(B(A(start)))'}
